In [1]:
#!/usr/bin/python

import sqlite3

import requests
# import eventlet
# eventlet.monkey_patch()
import string

import urllib.request
from bs4.element import Comment

from collections import Counter 
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
#word_tokenize accepts a string as an input, not a file. 

from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

from bs4 import BeautifulSoup 

import csv

from glob import glob; from os.path import expanduser

def write_to_webpage_csv():
    conn = sqlite3.connect('test.db')
    cursor = conn.cursor()
    cursor.execute("select * from webpage;")
    with open("webpage.csv", "w", newline='') as csv_file:  # Python 3 version    
#     with open("out.csv", "wb") as csv_file:              # Python 2 version
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow([i[0] for i in cursor.description]) # write headers
        csv_writer.writerows(cursor)
        
def write_to_links_csv():
    conn = sqlite3.connect('test.db')
    cursor = conn.cursor()
    cursor.execute("select * from link;")
    with open("link.csv", "w", newline='') as csv_file:  # Python 3 version    
#     with open("out.csv", "wb") as csv_file:              # Python 2 version
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow([i[0] for i in cursor.description]) # write headers
        csv_writer.writerows(cursor)

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

crawled = set ()
timed_out = set ()
error_urls = set ()
error_urls.add('http://vpsc.uiowa.edu/index.html')
links = []
url_words = []

def crawl_helper(url):
    global crawled
    crawled.add(url) 
    try:
        if(url not in error_urls and not url.startswith('http://vpsc.uiowa.edu/') 
           and not url.startswith('http://events.uiowa.edu/')
           and not url.startswith('https://ispo-apps.its.uiowa.edu/dports/')
           and not (url.endswith('.pdf') or url.endswith('.docx') or url.endswith('.xlsx') or url.endswith('.pptx'))):
            html = urllib.request.urlopen(url, timeout=15).read()
            top_10 = top_10_words(html)
            write_to_webpage(url, top_10)
            soup = BeautifulSoup(html)
            ahrefs =  soup.find_all('a', href=True)
    except:
        error_urls.add(url)
        return

    
    for a in ahrefs:
        # Filter out certain long timeout urls
        if(a['href'] not in error_urls and a['href'].startswith('http')):
            if('uiowa.edu' in a['href'] and (not a['href'].startswith('#')) 
               and (not a['href'].startswith('http://vpsc.uiowa.edu/')) 
               and (not a['href'].startswith('http://events.uiowa.edu/'))
               and (not a['href'].startswith('https://ispo-apps.its.uiowa.edu/dports/'))
               and not (a['href'].endswith('.pdf') or a['href'].endswith('.docx') or a['href'].endswith('.xlsx') or a['href'].endswith('.pptx'))):
                print("Found the URL:", a['href'])
                links.append([url,a['href']])
                write_to_link(url,a['href'])
                if(a['href'] not in crawled): 
                    crawl_helper(a['href'])
                else:
                    continue
            else:
                continue
        else:
            continue
                
        
def top_10_words(html):  
    data_set = text_from_html(html)

    # split() returns list of all the words in the string 
    split_it = data_set.split() 
    stemmed = []
    words = []


    ps = PorterStemmer() 

    for w in split_it: 
        stemmed.append(ps.stem(w)) 

    stop_words = set(stopwords.words('english')) 

    for word in stemmed: 
        if not word in stop_words:
            if not any(char in string.punctuation for char in word):
                words.append(word)
            else:
                continue
    # Pass the split_it list to instance of Counter class. 
    counter = Counter(words) 

    # most_common() produces k frequently encountered 
    # input values and their respective counts. 
    most_occur = counter.most_common(10) 

    print(most_occur)
    return most_occur


def crawl(url):
    crawl_helper(url)

def main():
#     drop_table_webpage()
#     create_table_webpage()
# #     drop_table_link()
#     create_table_link()
#     crawl('https://uiowa.edu')
    write_to_webpage_csv()
    write_to_links_csv()
    
def drop_table_webpage():    
    conn = sqlite3.connect('test.db')
    conn.execute("DROP TABLE WEBPAGE")
    print("Webpage table dropped successfully")
    conn.commit()
    conn.close()
    
def drop_table_link():    
    conn = sqlite3.connect('test.db')
    conn.execute("DROP TABLE LINK")
    print("Link table dropped successfully")
    conn.commit()
    conn.close()
    
def create_table_webpage():    
    conn = sqlite3.connect('test.db')
    conn.execute('''CREATE TABLE WEBPAGE
             (ID INTEGER PRIMARY KEY AUTOINCREMENT,
             URL TEXT    NOT NULL,
             WORD1            TEXT,
             WORD2            TEXT,
             WORD3            TEXT,
             WORD4            TEXT,
             WORD5            TEXT,
             WORD6            TEXT,
             WORD7            TEXT,
             WORD8            TEXT,
             WORD9            TEXT,
             WORD10            TEXT);''')
    print("Webpage table created successfully")
    conn.commit()
    conn.close()

def create_table_link():
    conn = sqlite3.connect('test.db')
    conn.execute('''CREATE TABLE LINK
             (ID INTEGER PRIMARY KEY AUTOINCREMENT,
             FROM_URL TEXT NOT NULL,
             TO_URL TEXT NOT NULL);''')
    print("Link table created successfully")
    conn.commit()
    conn.close()
    
def write_to_webpage(url, words):
    conn = sqlite3.connect('test.db')
    word_array = []
    for i in range (0,10):
        if i < len(words)-1:
            word_array.append(words[i][0])
        else:
            word_array.append("")
#     print(word1)
    conn.execute("INSERT INTO WEBPAGE(URL, WORD1, WORD2, WORD3, WORD4, WORD5, WORD6, WORD7, WORD8, WORD9, WORD10) VALUES (?,?,?,?,?,?,?,?,?,?,?)",(url, word_array[0], word_array[1], word_array[2], word_array[3], word_array[4], word_array[5], word_array[6], word_array[7], word_array[8], word_array[9]))
    print("Webpage record created successfully")
    conn.commit()
    conn.close()
    
def write_to_link(from_url, to_url):
    conn = sqlite3.connect('test.db')
    conn.execute("INSERT INTO LINK(FROM_URL, TO_URL) VALUES (?,?)",(from_url, to_url))
    print("Link record created successfully")
    conn.commit()
    conn.close()

if __name__ == '__main__':
    main()